<div style="background-image:url(instaseis_images/meschede-seismic-waves.png); padding: 10px 30px 20px 30px; background-size:cover; background-opacity:50%; border-radius:5px; background-position: 0px -200px">
<p style="float:right; margin-top:20px; padding: 20px 60px 0px 10px; background:rgba(255,255,255,0.75); border-radius:10px;">
<img width="400px" src=instaseis_images/logo.png>
</p>

<h1 style="color:#BBB; padding-bottom: 150px">Instaseis Tutorial</h1>

</div>

**Goal of the Practical:** To be able to calculate seismograms on a 1D background model with [Instaseis](http://www.instaseis.net), assuming the Green's function database computed with [AxiSEM](http://www.axisem.info) is provided.


**Approximate Required Time Investement:**

* [40 min] Python API intro and basic exercise (everybody should do this): use Instaseis to compute seismograms for the same earthquakes and stations as the last days.
* [30 - x min] Advanced exercises: some fun stuff to choose from

**Basic Tasks:**

* Calculate synthetics for the given events and stations

**Advanced Tasks:**

* Plot record section
* Finite Source, compare to point source solution

-----

Basic lines to set up the notebook and some paths.

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import obspy
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 8)

event_folder = os.path.join("data", "events")
stations_folder = os.path.join("data", "stations")
quakeml_folder = os.path.join(event_folder, "quakeml")
cmtsolutions_folder = os.path.join(event_folder, "cmtsolutions")
finite_source_folder = os.path.join(event_folder, "finite_source")

-----

## Basic Instaseis API Introduction

Please also have a look at our webpage - http://www.instaseis.net/ - where everything is documented extensively.

### Opening a Database

Before you can get going you have to (as always in Python) import `instaseis`.

In [ ]:
import instaseis

An Instaseis database must be opened before it can be used. Please keep in mind that this is by far the most expensive operation in Instaseis so you want to limit how often you do it. There are a couple of optional parameters here, for now the defaults are fine.

The database is at **./data/database**

In [ ]:
db = instaseis.open_db("data/database")

Some basic information about the loaded database can be reviewed by just printing it.

In [ ]:
print(db)

From this you can already glance a couple of aspects of the database used for this tutorial:

* uses ak135f as its 1D model
* is accurate for periods down to 20 seconds
* includes vertical and horizontal components
* sources can have depths ranging from 0 to 150 km
* one hour long seismograms

### Receivers and Sources

Instaseis calculates seismograms for any source and receiver pair. A receiver has coordinates and optionally network and station codes. Using a reciprocal database, all receivers are assumed to be at the same depth, i.e. usually at the Earth surface.

In [ ]:
rec = instaseis.Receiver(latitude=44.06238, longitude=10.59698,
                         network="IV", station="BDI")
print(rec)

Sources are naturally a bit more complex and Instaseis offers a variety of ways to define them. A straightforward way for earthquakes is to pass coordinates, moment as well as strike, dip and rake.

In [ ]:
src = instaseis.Source.from_strike_dip_rake(
    latitude=27.77, longitude=85.37, depth_in_m=12000.0,
    M0=1e+21, strike=32., dip=62., rake=90.)
print(src)

**Sidenode:** The moment tensor can be visualized using the Beachball function from obspy.imaging:

In [ ]:
from obspy.imaging.beachball import Beachball

mt = src.tensor / src.M0 # normalize the tensor to avoid problems in the plotting
Beachball(mt, size=200, linewidth=2, facecolor='b');

Now we are ready to extract synthetic seismograms from the database:

In [ ]:
st = db.get_seismograms(source=src, receiver=rec)
st.plot();

**Done** This is all you need for a basic usage of Instaseis!

## Basic Exercise

**Task:** Calculate three component synthetics for the stations and events used on Monday and today and save them on disc in an organized manner so that you can work with them tomorrow.

#### Notes

1. Receiver objects can also be created from StationXML, SEED, or STATIONS files as well as obpy inventories using `instaseis.Receiver.parse()`; see the [documentation](http://www.instaseis.net/source.html#receiver) for details.
2. Source objects can also be created from QuakeML, CMTSOLUTIONS, and in other ways using `instaseis.Source.parse()`; see the [documentation](http://www.instaseis.net/source.html#source) for details.
3. The `get_seismograms()` method has a couple of extra arguments:
  * `kind`: `displacement`, `velocity`, `acceleration`
  * `remove_source_shift`, `reconvolve_stf`, `dt`,
  
  ... see the [documentation](http://www.instaseis.net/instaseis.html#instaseis.base_instaseis_db.BaseInstaseisDB.get_seismograms) for details.
4. You can use the properties of the Receiver and Source objects to create usefull filenames.

### 1. Load Receivers

**reminder:** you can use ObsPy to load stations and plot a map:

In [ ]:
from obspy import read_inventory

station_filename = os.path.join(stations_folder, "all_stations.xml")

inventory = read_inventory(station_filename)
inventory.plot(projection="local", resolution="i");

This inventory can directly be used as input to generate `instaseis.Receiver` objects:

In [ ]:
receivers = instaseis.Receiver.parse(inventory)
for rec in receivers[:2]:
    print(rec)

**Alternatively**, instaseis can directly open the station xml or STATIONS file (but then you don't have the nice plot):

In [ ]:
receivers = instaseis.Receiver.parse(os.path.join(stations_folder, "all_stations.xml"))
print(receivers[0])
receivers = instaseis.Receiver.parse(os.path.join(stations_folder, "STATIONS"))
print(receivers[0])

### 2. Load Events
**reminder:** use ObsPy to load events from a QuakeML file containing all events and plot a map:

In [ ]:
import glob # provides iterator to loop over files

cat = obspy.core.event.Catalog()

for filename in glob.iglob(os.path.join(quakeml_folder, '*.xml')):
     cat += obspy.readEvents(filename)
        
print(cat)
print(instaseis.Source.parse(cat.events[0]))
cat.plot();

**Alternatively** load QuakeML or CMTSOLUTION files directly using `instaseis.Source.parse()` and store the sources in a list:

In [ ]:
sources = []

for filename in glob.iglob(os.path.join(quakeml_folder, '*.xml')):
    sources.append(instaseis.Source.parse(filename))
    
print(sources[0])

for filename in glob.iglob(os.path.join(cmtsolutions_folder, '*')):
    sources.append(instaseis.Source.parse(filename))

print(sources[0])

### 3. Extract Seismograms and Save to File

For the first solution using a ObsPy event catalog:

In [ ]:
dt = 1.0

for event in cat:
    src = instaseis.Source.parse(event)
    srcname = '%s_Mw_%3.1f' % (src.origin_time.date, src.moment_magnitude)
    for rec in receivers:
        # create a usefull filename
        recname = '%s_%s' % (rec.network, rec.station)
        filename = '%s_%s' % (recname, srcname)
        filename = filename.replace('.', '_')
        
        # extract seismograms using instaseis
        
        # write to miniseed files in the data_out folder. Write as MiniSEED due to multi
        # component support.


For the second solution use a list of sources:

In [ ]:
dt = 1.0

for src in sources:
    srcname = '%s_Mw_%3.1f' % (src.origin_time.date, src.moment_magnitude)
    for rec in receivers:
        # create a usefull filename
        recname = '%s_%s' % (rec.network, rec.station)
        filename = '%s_%s' % (recname, srcname)
        filename = filename.replace('.', '_')
        
        # extract seismograms using instaseis
        
        # write to miniseed files in the data_out folder. Write as MiniSEED due to multi
        # component support.


## Advanced Exercise 1: Plot Record Section

Use `Instaseis` to calculate a record section of your choice.

![image](./instaseis_images/record_section.png)

#### Notes

1. ObsPy Trace objects provide a function `times()` to conveniently get the time axis for plotting and the `normalize()` function to normalize seismograms to a common amplitude.
2. Use high pass filtering and deep sources if you want to enhance body waves
3. `np.linspace()` can help to generate equidistant stations
4. `obspy.taup.TauPyModel` can be used to compare with ray theoretical arrivals

### Advanced Exercise 2: Finite Source Effects

For earthquakes with Magnitude up to about 5 recorded at teleseismic distances, approximating the fault by a point source is a reasonable approach. However, for larger earthquakes with longer rupture duration this approximation is not valid anymore. In this exercise, you will compare the point source approximation with finite source solutions to understand its limitations.

For three of the earthquakes we use in this tutorial, USGS provides finite fault solutions: 
the recent event in [Nepal](http://earthquake.usgs.gov/earthquakes/eventpage/us20002926#scientific_finitefault),
the largest [aftershock](http://earthquake.usgs.gov/earthquakes/eventpage/us20002ejl#scientific_finitefault)
and the one in [Chile](http://earthquake.usgs.gov/earthquakes/eventpage/usc000nzvd#scientific_finitefault). This is the fault solution and slip as a function of time for the Nepal M7.9 event:

<p style="float: left; font-size: 9pt; text-align: center; width: 49%; margin-right: 1%; margin-bottom: 0.5em;"><img src="instaseis_images/finite_source_fault.png" style="width: 100%">Fault representation (image: USGS)</p><p style="float: right; font-size: 9pt; text-align: center; width: 49%; margin-right: 1%; margin-bottom: 0.5em;"><img src="instaseis_images/finite_source_stf.png" style="width: 100%">Source Time Function (image: USGS)</p><p style="clear: both;">

In Instaseis, a finite fault is represented as set of point sources, where each point source represents one of the fault patches with individual source time function. This functionality is provided by the `instaseis.FiniteSource` object [(see Documentation)](http://instaseis.net/source.html#instaseis.source.FiniteSource). It can be initialized in two ways: from a list of point sources, or more conveniently by reading \*.param files provided by USGS or standard rupture format (\*.srf) files (these can also be used in the GUI).

In [ ]:
finite_source = instaseis.FiniteSource.from_usgs_param_file(
    os.path.join(finite_source_folder,
                 "FINITE_SOURCE_2015_05_12__Mw_7_2_Nepal.param"))
print(finite_source)

A point source can be computed as a sum of all point sources weighted by their moment:

In [ ]:
finite_source.compute_centroid()
print(finite_source.CMT)

The hypo- and epicenter can be found as the fault patch that ruptures first:

In [ ]:
finite_source.find_hypocenter()
print('hypocenter latitude:', finite_source.hypocenter_latitude,
      'longitude:', finite_source.hypocenter_longitude,
      'depth:', finite_source.hypocenter_depth_in_m / 1e3)

**Task:** Compare the seismograms for three different representations of the source: 

* A point source with simple gaussian source time function (using CMTSOLUTION or quakeml files),
* the CMT solution using the more complex source time function provided by `finite_source.CMT`
* the full finite source solution using the `FiniteSource` object and `db.get_seismograms_finite_source()`

**Note:** First, you have to adapt the sampling of the source time functions in the finite source to the database, which works like this:

In [ ]:
# reloading finite source here to be sure to have a clean source time function
finite_source = instaseis.FiniteSource.from_usgs_param_file(
    os.path.join(finite_source_folder,
                 "FINITE_SOURCE_2015_04_25__Mw_7_9_Nepal.param"))

# prepare the source time functions to be at the same sampling as the database
# first use enough samples such that the lowpassed stf will still be correctly represented
nsamp = int(db.info.period / finite_source[0].dt) * 50
finite_source.resample_sliprate(dt=finite_source[0].dt, nsamp=nsamp)
# lowpass to avoid aliasing
finite_source.lp_sliprate(freq=1.0/db.info.period)
# finally resample to the sampling as the database
finite_source.resample_sliprate(dt=db.info.dt, nsamp=db.info.npts)

finite_source.compute_centroid()

**Solution**

In [ ]:
simple_source = instaseis.Source.parse(
    os.path.join(quakeml_folder, "GCMT_2015_04_25__Mw_7_9.xml"))


### Acknowledgements

Background picture at the very top is from Matthias Meschede.